# Aufgabe 7: AutoML

Unter AutoML versteht man...

https://www.ml4aad.org/automl/

###### (a) Wählen Sie ein AutoML Package. Begründen Sie Ihre Auswahl.

Wir haben uns für _H2O AutoML_ entschieden, weil es in Hinblick auf die Reproduzierbarkeit der Ergebnisse am leichtesten auf verschiedenen Plattformen installierbar ist.

Bei _auto-sklearn_ beispielsweise fehlten die Berechtigungen, um die benötigten Dependencies auf SDIL zu installieren.
Für andere AutoML-Bibliotheken wurde lediglich ein git-repo zur Verfügung gestellt, weshalb diese nicht einfach über eine `requirements.txt` installiert werden können.

Zudem fanden wir die Dokumentation zu _H2O AutoML_ auf den ersten Blick sehr übersichtlich und den Funktionsumfang sehr umfangreich.

###### (b) Führen Sie die Klassifikationsaufgabe von Aufgabe 3 mit AutoML durch. Vergleichen Sie die Ergebnisse mit den Ergebnissen aus Aufgabe 3.

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [79]:
# Import Libraries
import time
import multiprocessing

import pandas as pd
import numpy as np

from sklearn import svm
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

import h2o
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,31 mins 32 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,3 months and 27 days !!!
H2O cluster name:,H2O_from_python_uiuqb_97w3zp
H2O cluster total nodes:,1
H2O cluster free memory:,954 Mb
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster status:,"locked, healthy"


In [85]:
# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
iris = pd.read_csv(url, names=names)

In [74]:
# Split-out validation dataset
array = iris.values
X = array[:,0:4]
Y = array[:,4]
test_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

In [88]:
train_array = np.column_stack([X_train, y_train])
test_array = np.column_stack([X_test, y_test])

train = H2OFrame(python_obj=train_array, column_names=names)
test = H2OFrame(python_obj=test_array, column_names=names)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [89]:
# Identify predictors and response
x = names
y = "class"
x.remove(y)

In [90]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)                                 

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [91]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_1_AutoML_20190516_165455_model_2,0.00854701,0.0290967,0.08829,0.00779513
DeepLearning_grid_1_AutoML_20190516_165455_model_3,0.0175439,0.0583454,0.130914,0.0171384
StackedEnsemble_BestOfFamily_AutoML_20190516_165455,0.0258659,0.156888,0.169384,0.0286908
XRT_1_AutoML_20190516_165455,0.0258659,0.0630758,0.141359,0.0199824
DeepLearning_grid_1_AutoML_20190516_165455_model_1,0.0260909,0.143455,0.157654,0.0248548
DRF_1_AutoML_20190516_165455,0.0346379,0.0609861,0.139426,0.0194396
StackedEnsemble_AllModels_AutoML_20190516_165455,0.0346379,0.146685,0.163812,0.0268342
GLM_grid_1_AutoML_20190516_165455_model_1,0.0346379,0.0649054,0.139925,0.0195791
GBM_1_AutoML_20190516_165455,0.0346379,0.105274,0.160896,0.0258876
XGBoost_1_AutoML_20190516_165455,0.0346379,0.171388,0.186674,0.0348473


In [92]:
# The leader model is stored here
aml.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_1_AutoML_20190516_165455_model_2


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.007474681306671697
RMSE: 0.08645623925820332
LogLoss: 0.025204705450012653
Mean Per-Class Error: 0.008547008547008546
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
43.0,0.0,0.0,0.0,0 / 43
0.0,38.0,0.0,0.0,0 / 38
0.0,1.0,38.0,0.0256410,1 / 39
43.0,39.0,38.0,0.0083333,1 / 120


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9916667
2,1.0
3,1.0



ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.00779512787135134
RMSE: 0.08829002135774654
LogLoss: 0.029096689585768266
Mean Per-Class Error: 0.008547008547008546
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9916667,0.0117851,1.0,0.9583333,1.0,1.0,1.0
err,0.0083333,0.0117851,0.0,0.0416667,0.0,0.0,0.0
err_count,0.2,0.2828427,0.0,1.0,0.0,0.0,0.0
logloss,0.0290967,0.0231850,0.0035988,0.0796856,0.0011681,0.0568024,0.0042285
max_per_class_error,0.0285714,0.0404061,0.0,0.1428571,0.0,0.0,0.0
mean_per_class_accuracy,0.9904762,0.0134687,1.0,0.9523810,1.0,1.0,1.0
mean_per_class_error,0.0095238,0.0134687,0.0,0.0476191,0.0,0.0,0.0
mse,0.0077951,0.0075004,0.0002641,0.0272133,0.0000049,0.0112099,0.0002835
r2,0.9874415,0.0120686,0.9996831,0.9563375,0.9999919,0.9816043,0.9995907
rmse,0.0612290,0.0449785,0.0162514,0.1649645,0.0022158,0.1058766,0.0168367


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
,2019-05-16 16:57:23,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-05-16 16:57:24,1 min 12.438 sec,3287 obs/sec,10.0,1,1200.0,0.1315996,0.0637520,0.9746146,0.0333333
,2019-05-16 16:57:29,1 min 17.522 sec,5961 obs/sec,270.0,27,32400.0,0.0792858,0.0216034,0.9907856,0.0083333
,2019-05-16 16:57:34,1 min 22.606 sec,6623 obs/sec,580.0,58,69600.0,0.0741273,0.0146360,0.9919456,0.0083333
,2019-05-16 16:57:39,1 min 27.630 sec,7035 obs/sec,910.0,91,109200.0,0.0701612,0.0136194,0.9927845,0.0083333
,2019-05-16 16:57:41,1 min 29.567 sec,7153 obs/sec,1040.0,104,124800.0,0.0864562,0.0252047,0.9890436,0.0083333


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
petal-width,1.0,1.0,0.3595691
petal-length,0.8634027,0.8634027,0.3104530
sepal-width,0.5144761,0.5144761,0.1849897
sepal-length,0.4032277,0.4032277,0.1449882


In [96]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

predictions = aml.leader.predict(test).as_data_frame()['predict'].tolist()

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [97]:
y_test = test.as_data_frame()['class'].tolist()

In [98]:
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.8666666666666667
[[ 7  0  0]
 [ 0 10  2]
 [ 0  2  9]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         7
Iris-versicolor       0.83      0.83      0.83        12
 Iris-virginica       0.82      0.82      0.82        11

    avg / total       0.87      0.87      0.87        30



###### (c) Was ist Ihre Meinung zu AutoML?